# Table of Contents


- [**Setup**](#Setup): Runs any imports and other setup.
- [**Inputs swaption**](#Inputs-swaption): Defined the inputs for the model.
- [**Cash flows**](#Cash-flows): Determining the cash flows that Australian company pays and receives.
- [**Pricing swaption**](#Pricing-swaption):Calculating the fair price of the swaption using binomial option pricing model.
- [**Results**](#Results):Results after pricing the swaption.

# Setup

In [24]:
import numpy as np
import numpy_financial as npf
from datetime import date
from dataclasses import dataclass




# Inputs swaption

In [25]:
@dataclass
class SwapInputs:   
    #face value Australian Company
    face_value_Austral_company: int = 36000000
    #face value counter party
    face_value_counter_party: int = 40000000
    # pay fixed
    fixed_rate_Austral: float = 0.085
    # receive fixed
    fixed_rate_counter_party: float = 0.14
    # Foreign exchange 0.6USD/1AUD
    FX_USD_per_AUD: float = 0.6
    # risk free rate US
    rf_US: float = 0.09
    #risk free Australia
    rf_Australia: float = 0.04
    # Volatility 
    volatility: float = 0.55
    # maturity in years
    maturity_swaption: float = 1.5
    # number_steps 
    number_steps: int = 3

swap_data = SwapInputs()
swap_data


SwapInputs(face_value_Austral_company=36000000, face_value_counter_party=40000000, fixed_rate_Austral=0.085, fixed_rate_counter_party=0.14, FX_USD_per_AUD=0.6, rf_US=0.09, rf_Australia=0.04, volatility=0.55, maturity_swaption=1.5, number_steps=3)

## Calculate excact days between dates

In [26]:
t1_days = date(2026, 6, 30) - date(2025, 12,  31)
t1=t1_days.days
t2_days = date(2026, 12, 31) - date(2026, 6,  30)
t2 = t2_days.days
t3_days = date(2027, 6, 30) - date(2026, 12,  31)
t3 = t3_days.days
t4_days = date(2027, 12, 31) - date(2027, 6,  30)
t4 = t4_days.days
t5_days = date(2028, 6, 30) - date(2027, 12,  31)
t5 = t5_days.days
# 2028/12/31 falls on Sunday and the next working day is in a new month,
#then the payment t6 will be made on the previous business day 2028/12/29,
#which is typically the last working day before the weekend.
t6_days = date(2028, 12, 29) - date(2028, 6,  30)
t6 = t6_days.days
days = [t1,t2,t3,t4,t5,t6]
days


[181, 184, 181, 184, 182, 182]

### Calculate coupon periods as a percentage based on convention

In [27]:
coupon_periods = [t1/365,t2/365,t3/365,t4/365,t5/365,t6/365];coupon_periods

[0.4958904109589041,
 0.5041095890410959,
 0.4958904109589041,
 0.5041095890410959,
 0.4986301369863014,
 0.4986301369863014]

# Cash flows
### Australian company pays in millions(cash flows-USD)

In [28]:
cash_flows_pays = []
for i in range(len(coupon_periods)):
    cash_flow = swap_data.face_value_Austral_company*swap_data.fixed_rate_Austral*coupon_periods[i]
    cash_flows_pays.append(cash_flow)
        
cash_flows_pays[5] =  cash_flows_pays[5] + swap_data.face_value_Austral_company
print(f'The cash flows that Australian company will pay each period are:\n')
print(cash_flows_pays)

The cash flows that Australian company will pay each period are:

[1517424.6575342466, 1542575.3424657534, 1517424.6575342466, 1542575.3424657534, 1525808.2191780822, 37525808.21917808]


### Present value of Australian company payments

In [29]:
def pv_bond(cash_flows_list,r):
    present_value_bond = 0
    for i in range(len(cash_flows_list)): 
        present_value_cash_flow = cash_flows_list[i] / (1 + r)**(i+1)
        print(f'Present Value T{i+1}: {present_value_cash_flow}')
        present_value_bond += present_value_cash_flow
    return present_value_bond

In [30]:
bond_pays_USD = pv_bond(cash_flows_pays,swap_data.rf_US/2);bond_pays_USD
print(f'\nThe present value of cash flows that Australian company pays is: \033[1m{bond_pays_USD}\033[0m million in USD.')

Present Value T1: 1452081.0119944944
Present Value T2: 1412582.4431361496
Present Value T3: 1329714.074306444
Present Value T4: 1293544.0517718457
Present Value T5: 1224386.402238797
Present Value T6: 28815908.206950642

The present value of cash flows that Australian company pays is: 35528216.19039837 million in USD.


### Australian Company receives in millions(cash flow- AUD)


In [31]:
cash_flows_receives = []
for i in range(len(coupon_periods)):
    cash_flow = swap_data.face_value_counter_party*swap_data.fixed_rate_counter_party*coupon_periods[i]
    cash_flows_receives.append(cash_flow)
        
cash_flows_receives[5] =  cash_flows_receives[5] + swap_data.face_value_counter_party
print(f'The cash flows that the Australian company will receive are:\n' )
print(cash_flows_receives)

The cash flows that the Australian company will receive are:

[2776986.3013698636, 2823013.698630138, 2776986.3013698636, 2823013.698630138, 2792328.767123288, 42792328.76712329]


### Present value of counter party payments in AUD

In [32]:
bond_receives_in_AUD = pv_bond(cash_flows_receives,swap_data.rf_Australia/2);bond_receives_in_AUD
print(f'\nThe present value that Australian company receive is \033[1m{bond_receives_in_AUD}\033[0m million in AUD. ')

Present Value T1: 2722535.5895782975
Present Value T2: 2713392.636130467
Present Value T3: 2616816.214512012
Present Value T4: 2608028.2930896454
Present Value T5: 2529098.1955579463
Present Value T6: 37998363.32230841

The present value that Australian company receive is 51188234.25117678 million in AUD. 


# Pricing swaption 

In [33]:
def binomial_tree_pricing(sigma, r, S_0, B, K, T, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    q = (1+r * dt) - d / (u - d)
    
    
    fx = np.zeros((N+1, N+1))

    # Create the tree for the foreign exchange
    for i in range(N+1):
        for j in range(i+1):
            fx[i, j] = S_0 * (d ** (i - j)) * (u ** j)

    payoff = np.zeros((N+1, N+1))
    
    #   calculate the payoff for the swaption at expiry
    for j in range(N+1):
        if fx[N, j] * B - K > 0:# take the max[0,receive*foreign exchange-pay]
            payoff[N, j] = fx[N , j] * B - K
    

    # now iterate the binomial tree backward to obtain the  prices.
    # calculated the expected value for each node and discounted
    for i in range(N-1, -1, -1):
        for j in range(i,-1,-1):
            payoff[i, j] = (1 / (1 + dt*r)) * (q * payoff[i+1, j+1] + (1 - q) * payoff[i+1, j])

    return fx, payoff


In [34]:
fx , payoff = binomial_tree_pricing(swap_data.volatility,
                                swap_data.rf_US,
                                swap_data.FX_USD_per_AUD,
                                bond_receives_in_AUD,
                                bond_pays_USD,
                                swap_data.maturity_swaption,
                                swap_data.number_steps)

In [35]:
print(f'\033[1mThe tree for the foreign exchange is\033[0m\n')
print(fx)

The tree for the foreign exchange is

[[0.6        0.         0.         0.        ]
 [0.40667768 0.88522193 0.         0.        ]
 [0.27564456 0.6        1.30602978 0.        ]
 [0.18683081 0.40667768 0.88522193 1.92687701]]


# Results

In [36]:
print(payoff)

[[ 1199798.79417475        0.                0.                0.        ]
 [  341331.05527503  5016351.04757808        0.                0.        ]
 [       0.          1827520.92046796 19322191.34190307        0.        ]
 [       0.                0.          9784731.45977585 63105215.80684482]]


In [37]:
print(f'The fair price of the swaption is  \033[1m{payoff[0,0]}\033[0m million USD$.')

The fair price of the swaption is  1199798.794174753 million USD$.
